In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pds
import pathlib 
import math
import cmath
from scipy import stats
from scipy.constants import k
from copy import deepcopy
import requests
from bs4 import BeautifulSoup
import pandas as pd
from geopy.geocoders import Nominatim
import random
import folium
from datetime import datetime
import time
import responses
import googlemaps
import numpy as np
%matplotlib inline
import pickle

In [5]:
with open("API_key.txt", "r") as f:
    GMAP_API_KEY = f.read()
"""
This calls the Google Maps API on all pairwise combinations of a supplied list of M addresses, and computes an MxM matrix for distances and times.

Input: List of addresses

Output: Distance Matrix, Time Matrix, in seconds and in meters
"""

def getGoogleTimeDistance(addresses):
    gmaps = googlemaps.Client(key=GMAP_API_KEY)

    curDate = datetime.now()

    time_matrix = np.array([[0] * len(addresses)] * len(addresses))
    distance_matrix = np.array([[0] * len(addresses)] * len(addresses))

    # Iterate through all possible address combinations.
    for i in range(0, len(addresses)):
        for j in range(0, len(addresses)):

            # Do not calculate distance for the same address
            if i == j:
                continue
           
            # Call Google API and get response.
            result = gmaps.distance_matrix(addresses[i], addresses[j], mode='driving', units='metric',traffic_model='best_guess', departure_time=curDate)

            # Index to relevant part of response JSON File.
            curElement = result['rows'][0]['elements'][0]

            # Update matrices
            distance_matrix[i][j] = curElement['distance']['value']
            time_matrix[i][j] = curElement['duration']['value']

    return time_matrix, distance_matrix

def coords_from_address(address):
    """
    inputs: str address
    outputs: tuple[float]  result_loc
    description: converts address to into latitude and longitude
    """
    geolocator = Nominatim(user_agent = "test")
    location = geolocator.geocode(address)
    result_loc = (location.latitude, location.longitude)
    return result_loc

def write_gps_to_csv(gps_list, filename):
    df = pd.DataFrame(gps_list, columns=['Latitude', 'Longitude'])
    df.to_csv(filename, index=False)
    
#write_gps_to_csv(all_coords, "coords.csv")

In [3]:
coords1 = []
coords2 = []
coords3 = []
addr1 = [
    "2515 Bank St, Ottawa",
    "30 Haxby Private, Ottawa",
    "116 Sai Crescent",
    "214 Plymouth St, Ottawa",
    "266 Lorry Greenberg Dr, Ottawa",
    "800 King Edward Ave, Ottawa",
    "2210 Bank St, Ottawa",
    "2161 Thurston Dr, Ottawa"]

addr2 =[
    "23 Ella Street, Ottawa",
    "11 Oblats Avenue, Ottawa",
    "422 Byron Avenue, Ottawa",
    "1680 Fisher Avenue, Ottawa",
    "825 Cahill Drive, Ottawa",
    "1125 Colonel By Dr, Ottawa"]
    
addr3 = [
    "54 Hawk Crescent, Ottawa",
    "1380 Prince of Wales Drive, Ottawa",
    "75 Laurier Ave. E, Ottawa, Ontario, Canada",
    "14 Waller St, Ottawa, Ontario, Canada",
    "363 Rideau St, Ottawa, Ontario, Canada",
    "2701 Riverside Dr, Ottawa"
]

In [ ]:
for addr in addr1:
    print(addr)
    coords1.append(coords_from_address(addr))
    time.sleep(1)
    
for addr in addr2:
    print(addr)
    coords2.append(coords_from_address(addr))
    time.sleep(1)

for addr in addr3:
    print(addr)
    coords3.append(coords_from_address(addr))
    time.sleep(1)
all_coords = coords1+coords2+coords3


In [31]:
T_mat, D_mat = getGoogleTimeDistance(all_addr)


(45.4193955, -75.678723)

In [17]:
filename = "T_mat"
fileObject = open(filename, 'wb')

pickle.dump(T_mat, fileObject)
fileObject.close()

In [19]:
fileObject2 = open("T_mat", 'wb')
fileObject2.close()

In [29]:
with open('T_mat','wb') as f1: 
    pickle.dump(T_mat, f1)

with open('T_mat','rb') as f2:
    new_T_mat = pickle.load(f2)

In [27]:
with open('D_mat','wb') as f1: 
    pickle.dump(D_mat, f1)

with open('D_mat','rb') as f2:
    new_D_mat = pickle.load(f2)

In [30]:
new_T_mat

array([[   0,  359,  494,  779,  344, 1129,  307,  435,  713,  924, 1194,
         828,  409,  752,  405,  667, 1092, 1224, 1229,  526],
       [ 346,    0,  443,  920,  184, 1170,  348,  279,  855,  897, 1299,
         957,  551,  893,  547,  796, 1133, 1265, 1360,  661],
       [ 472,  418,    0,  879,  385,  907,  608,  297, 1013,  942, 1069,
        1128,  709, 1052,  705,  967,  870, 1002, 1042,  826],
       [ 788,  959, 1020,    0,  971,  769,  815,  943,  311,  547,  434,
         656,  716,  471,  712,  543,  718,  767,  782,  486],
       [ 245,  176,  401,  852,    0, 1070,  381,  237,  787,  967, 1267,
         874,  482,  825,  478,  741, 1033, 1165, 1205,  599],
       [1022, 1039,  807,  280,  940,    0, 1082,  830,  420,  224,  470,
         830,  826,  580,  822,  789,  312,  557,  392,  596],
       [ 308,  346,  641,  705,  426, 1210,    0,  521,  640,  842, 1120,
         754,  335,  678,  332,  594, 1159, 1244, 1305,  453],
       [ 413,  275,  335,  908,  243,  89

In [25]:

def distance_matrix(addresses):
    """
    inputs: list[str] addresses
    outputs: dist_mat nparray
    description: returns an array that calculates the distance to travel from one address to all other addresses, for all addresses
    """
    pass

def time_matrix(addresses):
    """
    inputs: list[str] addresses
    outputs: time_mat nparray
    description: returns an array that calculates the travel time from one address to all other addresses, for all addresses
    """
    pass

def cost_function(start_idx, end_idx, distance_matrix, lookup_tables):
    """
    inputs: int start_idx, int end_idx, nparray distance_matrix, list[str] lookup_tables
    outputs: float cost
    description: calculates the cost to go from start_idx to end_idx. These idx correspond to distance matrix.
    cost also factors in weather and traffic, information could be obtained from a lookup table
    lookup_tables is filepath to csv file, which would be read to generate our lookup table
    """
    pass

def cost_matrix(distance_matrix, time_matrix, lookup_tables):
    """
    inputs: nparray distance_matrix, nparray time_matrix, str lookup_tables 
    outputs: nparray cost_mat
    description: returns an array that calculates the cost to go from one node to another, for all nodes
    """
    pass

def cluster_coords(coordinates, N_trucks):
    """
    inputs: list coordinates, int N_trucks
    outputs: list[list] grouped_coords
    description: groups the coordinates into n_clusters, where N_trucks = n_clusters
    grouped_coords is a list of lists containing coordinates, ex: grouped_coords[0] = list of all coordinates in cluster 0
    """
    pass

def sTSP(cost_matrix):
    """
    inputs: nparray cost_matrix
    outputs: list[tuple]
    description: calculates optimal sequence for a delivery truck
    """
    
def plot_route(travel_seq):
    """
    inputs: list[tuple] travel_seq
    outputs: none
    description: visualizes travel route on folium
    """